<a href="https://colab.research.google.com/github/adinotfound11/ModelCounting/blob/main/Tic_tac_toe_Z3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install z3-solver

from z3 import *

board_size = 3
Cells = [[Int(f"Cell_{i}_{j}") for j in range(board_size)] for i in range(board_size)]
solver = Solver()

# Each cell can be 0 (empty), 1 (X), or 2 (O)
for i in range(board_size):
    for j in range(board_size):
        solver.add(Cells[i][j] >= 0, Cells[i][j] <= 2)

num_X = Sum([If(Cells[i][j] == 1, 1, 0) for i in range(board_size) for j in range(board_size)])
num_O = Sum([If(Cells[i][j] == 2, 1, 0) for i in range(board_size) for j in range(board_size)])

# The number of X's and O's should differ by at most 1, and X goes first
solver.add(num_X - num_O <= 1)
solver.add(num_O - num_X <= 0)

def is_winner(player_value):
    row_wins = [And([Cells[i][j] == player_value for j in range(board_size)]) for i in range(board_size)]
    col_wins = [And([Cells[i][j] == player_value for i in range(board_size)]) for j in range(board_size)]
    diag1_win = And([Cells[i][i] == player_value for i in range(board_size)])
    diag2_win = And([Cells[i][board_size - i - 1] == player_value for i in range(board_size)])
    return Or(*row_wins, *col_wins, diag1_win, diag2_win)

# Both players cannot win simultaneously
solver.add(Not(And(is_winner(1), is_winner(2))))

# If X wins, X has played one more move than O
solver.add(Implies(is_winner(1), num_X == num_O + 1))

# If O wins, X and O have played the same number of moves
solver.add(Implies(is_winner(2), num_X == num_O))

def count_moves_and_difficulty():
    total_moves = 0
    x_wins = 0
    o_wins = 0

    while solver.check() == sat:
        model = solver.model()

        x_win = is_winner(1)
        o_win = is_winner(2)

        if is_true(model.evaluate(x_win)):
            x_wins += 1
        if is_true(model.evaluate(o_win)):
            o_wins += 1

        total_moves += 1

        # Exclude the current model from future iterations
        current_model_constraints = []
        for i in range(board_size):
            for j in range(board_size):
                val = model.evaluate(Cells[i][j])
                current_model_constraints.append(Cells[i][j] == val)
        solver.add(Not(And(current_model_constraints)))

    return total_moves, x_wins, o_wins

total_moves, x_wins, o_wins = count_moves_and_difficulty()

print(f"Total number of valid configurations: {total_moves}")
print(f"Total number of X win options: {x_wins}")
print(f"Total number of O win options: {o_wins}")

if total_moves > 0:
    difficulty_o = 1 - (o_wins / total_moves)
    difficulty_x = 1 - (x_wins / total_moves)
else:
    difficulty_o = difficulty_x = None

print(f"Difficulty for O: {difficulty_o}")
print(f"Difficulty for X: {difficulty_x}")


Total number of valid configurations: 5478
Total number of X win options: 626
Total number of O win options: 316
Difficulty for O: 0.9423147133990507
Difficulty for X: 0.8857247170500182


In [ ]:
!pip install z3-solver

from z3 import *
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib import animation
from IPython.display import HTML

# Define the grid dimensions and parameters
rows, cols = 2, 4
moves = 5
initial_position = [0, 0]
goal_position = [0, 3]

# Define enemy positions (static positions on the map)
enemy_positions = [(0, 1), (1, 3)]  # Example enemy positions

# Characters' movement abilities
movement_rules = {
    'Tank': [(0, 0), (0, -1), (0, 1)],       # Can stay or move left/right within the same row
    'Agility': [(0, 0), (0, -1), (0, 1),     # Can stay or move left/right
                (-1, 0), (1, 0)],            # Can move up/down
}

# Initialize a dictionary to hold variables and solvers for each character
characters = {}

# Function to initialize character variables
def initialize_character(name):
    position_vars = [[[Bool(f"{name}_{i}_{j}_{t}") for t in range(moves + 1)]
                      for j in range(cols)] for i in range(rows)]
    solver = Solver()
    characters[name] = {
        'vars': position_vars,
        'solver': solver,
    }

# Function to add constraints for a character
def add_constraints(name, movement_directions, avoid_enemies=False):
    vars = characters[name]['vars']
    solver = characters[name]['solver']

    # Constraint 1: Start position at initial position at time 0
    solver.add(vars[initial_position[0]][initial_position[1]][0] == True)

    # Constraint 2: The character must be in exactly one position at each time step
    for t in range(moves + 1):
        solver.add(Sum([If(vars[i][j][t], 1, 0)
                        for i in range(rows) for j in range(cols)]) == 1)
        # If the character must avoid enemies, add constraints to avoid enemy positions
        if avoid_enemies:
            for (ei, ej) in enemy_positions:
                solver.add(vars[ei][ej][t] == False)

    # Constraint 3: Movement constraints based on allowed directions
    for t in range(1, moves + 1):
        for i in range(rows):
            for j in range(cols):
                previous_positions = []
                for dx, dy in movement_directions:
                    prev_i, prev_j = i + dx, j + dy
                    if 0 <= prev_i < rows and 0 <= prev_j < cols:
                        previous_positions.append(vars[prev_i][prev_j][t - 1])
                if previous_positions:
                    solver.add(Implies(vars[i][j][t], Or(previous_positions)))
                else:
                    solver.add(vars[i][j][t] == False)

    # Constraint 4: The character should reach the goal position in 'moves' moves or less
    goal_positions = [vars[goal_position[0]][goal_position[1]][t]
                      for t in range(moves + 1)]
    solver.add(Or(goal_positions))

# Function to find paths for a character
def find_paths(name, max_paths=1):
    vars = characters[name]['vars']
    solver = characters[name]['solver']
    counter = 0
    paths = []
    local_solver = Solver()
    local_solver.add(solver.assertions())
    while local_solver.check() == sat:
        counter += 1
        model = local_solver.model()
        path = []
        for t in range(moves + 1):
            for i in range(rows):
                for j in range(cols):
                    if is_true(model.evaluate(vars[i][j][t])):
                        path.append((i, j))
                        break
        paths.append(path)
        # Exclude the current solution from future searches
        local_solver.add(Not(And([vars[i][j][t] == model.evaluate(vars[i][j][t])
                                  for i in range(rows) for j in range(cols)
                                  for t in range(moves + 1)])))
        if counter >= max_paths:
            print(f"Found {max_paths} paths for {name}.")
            break
    print(f"Total number of valid paths for {name}: {counter}")
    return paths

# Function to visualize paths using matplotlib animation
def visualize_paths(name, paths, symbol='C'):
    for idx, path in enumerate(paths, 1):
        print(f"\n{name} Path {idx}:")
        fig, ax = plt.subplots(figsize=(cols, rows))
        ax.set_xlim(0, cols)
        ax.set_ylim(0, rows)
        ax.set_xticks(range(cols+1))
        ax.set_yticks(range(rows+1))
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.grid(True)
        ax.invert_yaxis()

        # Draw enemies
        enemy_patches = []
        for (ei, ej) in enemy_positions:
            rect = patches.Rectangle((ej, ei), 1, 1, linewidth=1, edgecolor='black', facecolor='red')
            enemy_patches.append(rect)
            ax.add_patch(rect)

        # Draw goal position
        goal_rect = patches.Rectangle((goal_position[1], goal_position[0]), 1, 1, linewidth=2, edgecolor='green', facecolor='none', linestyle='--')
        ax.add_patch(goal_rect)

        # Character patch
        character_patch = patches.Rectangle((0, 0), 1, 1, linewidth=1, edgecolor='black', facecolor='blue')
        ax.add_patch(character_patch)

        def init():
            character_patch.set_xy((-1, -1))
            return [character_patch] + enemy_patches + [goal_rect]

        def animate(t):
            ci, cj = path[t]
            character_patch.set_xy((cj, ci))
            if (ci, cj) == tuple(goal_position):
                ax.set_title(f'Time {t}: GOAL', color='green', fontsize=16, fontweight='bold')
            else:
                ax.set_title(f'Time {t}')
            return [character_patch] + enemy_patches + [goal_rect]

        anim = animation.FuncAnimation(fig, animate, init_func=init, frames=len(path), interval=1000, blit=True)
        plt.close(fig)
        display(HTML(anim.to_jshtml()))

# Initialize characters
initialize_character('Tank')
initialize_character('Agility')

# Add constraints for each character
add_constraints('Tank', movement_rules['Tank'], avoid_enemies=False)
add_constraints('Agility', movement_rules['Agility'], avoid_enemies=True)

# Find paths for each character
tank_paths = find_paths('Tank', max_paths=3)       # Adjust max_paths as needed
agility_paths = find_paths('Agility', max_paths=3) # Adjust max_paths as needed

# Visualize paths for each character
print("\nTank Paths:")
visualize_paths('Tank', tank_paths, symbol='T')

print("\nAgility Character Paths:")
visualize_paths('Agility', agility_paths, symbol='A')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.1/28.1 MB 25.0 MB/s eta 0:00:00
Found 3 paths for Tank.
Total number of valid paths for Tank: 3
Total number of valid paths for Agility: 1

Tank Paths:

Tank Path 1:



Tank Path 2:



Tank Path 3:



Agility Character Paths:

Agility Path 1:
